In [1]:
import mechwolf as mw

# create components
a = mw.Vessel(name="a", description="nothing")
b = mw.Vessel(name="b", description="nothing")
c = mw.Vessel(name="c", description="nothing")

pump = mw.DummyPump(name="Dummy pump")


test = mw.DummySensor(name="test")
test2 = mw.DummySensor(name="test2")
test3 = mw.DummySensor(name="test3")
test4 = mw.DummySensor(name="test4")

tube = mw.Tube("1 foot", "1/16 in", "2/16 in", "PVC")

# create apparatus
A = mw.Apparatus()
A.add([a, b, c], pump, tube)
A.add(pump, test, tube)
A.add(pump, test2, tube)
A.add(pump, test3, tube)
A.add(pump, test4, tube)

P = mw.Protocol(A, duration="auto")
P.add(pump, rate="5 mL/min", start="0 seconds", stop="15 secs")
P.add(pump, rate="10 mL/min",start="15 seconds", stop="30 secs")
P.add(test, rate="5 Hz", start="4 secs", stop="30 secs")
P.add(test2, rate="5 Hz", start="4.1 secs", stop="30 secs")
P.add(test3, rate="5 Hz", start="4.1 secs", stop="30 secs")
P.add(test4, rate="5 Hz", start="4.1 secs", stop="30 secs")

In [2]:
#Returns immediately with an Experiment object that will update as it gets new data
E = P.execute()